# Scoreboard

From pool play run results we want to see scoreboard table

In [1]:
import pandas as pd
import numpy as np

In [2]:
pool_play_run_dir = 'pool_play_results/28-10-2020-05-30-10/'

In [3]:
rdf = pd.read_csv(pool_play_run_dir + 'results.csv')
print(rdf.shape)

(100, 9)


Want to remove TIMEOUT ones

In [4]:
rdf = rdf[rdf['right_status'] == 'DONE']
rdf = rdf[rdf['left_status'] == 'DONE']
print(rdf.shape)

(100, 9)


for agent, we calculate number of wins and losses, and goals for and goals against

In [5]:
agents = list(set(list(rdf.left_agent.unique()) + list(rdf.right_agent.unique())))

In [6]:
agents

['tunable-baseline-bot/submission_v6.py',
 'gfootball-with-memory-patterns/submission_v15.py',
 'best-open-rules-bot/submission_v2.py',
 'gfootball-with-memory-patterns/submission_v28.py',
 'gfootball-with-memory-patterns/submission_v32.py']

In [7]:
result_list = []

for agent in agents:
    left_df = rdf[rdf.left_agent == agent].reset_index(drop=True)
    
    # calculate num_wins, num_losses, num_ties
    left_df['num_wins'] = np.where(left_df['left_score'] > left_df['right_score'], 1, 0)
    left_df['num_losses'] = np.where(left_df['left_score'] < left_df['right_score'], 1, 0)
    left_df['num_ties'] = np.where(left_df['left_score'] == left_df['right_score'], 1, 0)
    
    games_played = len(left_df)
    goals_for = left_df.left_score.sum()
    goals_against = left_df.right_score.sum()
    num_wins = left_df.num_wins.sum()
    num_losses = left_df.num_losses.sum()
    num_ties = left_df.num_ties.sum()
    
    
    right_df = rdf[rdf.right_agent == agent].reset_index(drop=True)
    
    # calculate num_wins, num_losses, num_ties
    right_df['num_wins'] = np.where(right_df['right_score'] > right_df['left_score'], 1, 0)
    right_df['num_losses'] = np.where(right_df['right_score'] < right_df['left_score'], 1, 0)
    right_df['num_ties'] = np.where(right_df['right_score'] == right_df['left_score'], 1, 0)
    
    games_played = games_played + len(right_df)
    goals_for = goals_for + right_df.right_score.sum()
    goals_against = goals_against + right_df.left_score.sum()
    num_wins = num_wins + right_df.num_wins.sum()
    num_losses = num_losses + right_df.num_losses.sum()
    num_ties = num_ties + right_df.num_ties.sum()
    
    result_list.append([agent, games_played, num_wins, num_losses, num_ties, goals_for, goals_against])
    
fdf = pd.DataFrame(result_list, columns = ['agent', 'games_played', 'num_wins', 'num_losses', 'num_ties',
                                           'goals_for', 'goals_against'])

fdf['num_points'] = fdf['num_wins']*3 + fdf['num_ties']*1

fdf = fdf.sort_values('num_points', ascending=False)

In [8]:
fdf

,agent,games_played,num_wins,num_losses,num_ties,goals_for,goals_against,num_points
4,gfootball-with-memory-patterns/submission_v32.py,40,29,7,4,127,66,91
3,gfootball-with-memory-patterns/submission_v28.py,40,17,14,9,82,75,60
1,gfootball-with-memory-patterns/submission_v15.py,40,16,17,7,101,108,55
2,best-open-rules-bot/submission_v2.py,40,14,15,11,82,80,53
0,tunable-baseline-bot/submission_v6.py,40,6,29,5,66,129,23


## Testing

In [9]:
rdf.left_agent.value_counts()

tunable-baseline-bot/submission_v6.py               40
best-open-rules-bot/submission_v2.py                30
gfootball-with-memory-patterns/submission_v15.py    20
gfootball-with-memory-patterns/submission_v28.py    10
Name: left_agent, dtype: int64

In [10]:
rdf.right_agent.value_counts()

gfootball-with-memory-patterns/submission_v32.py    40
gfootball-with-memory-patterns/submission_v28.py    30
gfootball-with-memory-patterns/submission_v15.py    20
best-open-rules-bot/submission_v2.py                10
Name: right_agent, dtype: int64